# Current plots for Vshift (1, 5, 10, 67, 68) mV.

In [29]:
%matplotlib inline
import numpy as np
import pandas as pd
from parameters import *
from tau_ch_plot_study import *
from bokeh.io import output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

In [37]:
TOOLS = 'box_zoom,box_select,crosshair,reset,hover'

In [31]:
VSHIFT = [1e-3, 5e-3, 10e-3, 67e-3, 68e-3]
tau_multiplier = 1
f = 0
gmbar_Na = 120e-3 #next step
gmbar_K = 36e-3 #next step
E_Na = .05 # reversal potentail.
E_K = -.08
vmin, vmax, vdivs = get_voltage_scales()
v_values = np.linspace(vmin, vmax, vdivs)

def get_alpha_beta_params(parameters):
    return (parameters[:5], parameters[5:])

def compute_alpha_beta(v, A, B, C, D, F):
    C = 1 if B == 0 else -1
    return (A+B*v)/(C+ np.exp((D+v)/F))

In [32]:
def compute_data(vshift):
    global tau_multiplier, f
    K_n_params = get_k_n_params(VSHIFT=vshift, tau=tau_multiplier, f=f)
    Na_h_params = get_na_h_params(VSHIFT=vshift, tau=tau_multiplier, f=f)
    Na_m_params = get_na_m_params(VSHIFT=vshift, tau=tau_multiplier, f=f)
    data = pd.DataFrame(v_values, columns=['voltage'])
    data['k_n_alpha'] = data['voltage'].apply(compute_alpha_beta, 
                                          args=get_alpha_beta_params(K_n_params)[0])
    data['k_n_beta'] = data['voltage'].apply(compute_alpha_beta, 
                                         args=get_alpha_beta_params(K_n_params)[1])
    data['na_h_alpha'] = data['voltage'].apply(compute_alpha_beta,
                                           args=get_alpha_beta_params(Na_h_params)[0])
    data['na_h_beta'] = data['voltage'].apply(compute_alpha_beta, 
                                            args=get_alpha_beta_params(Na_h_params)[1])
    data['na_m_alpha'] = data['voltage'].apply(compute_alpha_beta, 
                                           args=get_alpha_beta_params(Na_m_params)[0])
    data['na_m_beta'] = data['voltage'].apply(compute_alpha_beta, 
                                         args=get_alpha_beta_params(Na_m_params)[1])
    data['k_n_ss'] = data['k_n_alpha']/(data['k_n_alpha']+data['k_n_beta'])
    data['na_m_ss'] = data['na_m_alpha']/(data['na_m_alpha']+data['na_m_beta'])
    data['na_h_ss'] = data['na_h_alpha']/(data['na_h_alpha']+data['na_h_beta'])
    data['m_cube'] = data['na_m_ss']**3
    data['h_cube'] = data['na_h_ss']**3
    data['n_forth'] = data['k_n_ss']**4
    data['g_na'] = data['m_cube']*gmbar_Na*1
    data['g_k'] = data['n_forth']*gmbar_K*1
    data['na_v'] = data['voltage'] - E_Na
    data['k_v'] = data['voltage'] - E_K
    data['i_na'] = data['g_na'] * data['na_v']
    data['i_k'] = data['g_k'] * data['k_v']
    data['i_total'] = data['i_na'] + data['i_k']
    return data

In [38]:
def get_plot(data):
    p = figure(title="VI characteristics", x_axis_label='voltage', y_axis_label='current', tools=TOOLS)
    p.line(data['voltage'], data['i_na'], line_color='blue', legend='i_na')
    p.line(data['voltage'], data['i_k'], line_color='green', legend='i_k')
    p.line(data['voltage'], data['i_total'], line_color='red', legend='i_total')
    return p

In [34]:
data_frames= [compute_data(v) for v in VSHIFT]

In [39]:
plots = [get_plot(data) for data in data_frames]

#### In the below graph I_total cross zero current line at -61mV
#### This the voltage point where cell start to generate action potential spike (start of the raise phase)
### 1mV Vshift

In [40]:
show(plots[0])

##### -55mV is the treshold potential where the cell starts to spike
#### 5mV Vshift.

In [25]:
show(plots[1])

#### -48mV the start of action potential for the cell starts
### 10mV vshift

In [26]:
show(plots[2])

### 67mV Vshift No V threshold for action potentail the cell is 
#### what positive current mean ??? is cell being hyperpolarized or depolarized.
### postive current is leaving the cell, cell is hyper polarizing.
### negative current is leaving the cell, cell is depolarizing. 

In [27]:
show(plots[3])

In [28]:
show(plots[4])

#### Points after discussion. (important observation after plotting)

Never say charging curve about raise phase of action potential
action potential is a whole hill part in simulation.

give -ve vshift only to K channel and try to see the peaks in simulation before and after injection current.
Which means you are setting the resting potential close to threshold of action potential this resulting in cell to spike.

write to function to find zero crossing current curve and set limits to the graph.